# Chinese (Simplified) Tokenization Model - Experiments - TODO

In [19]:
import os, sys
cwd = os.getcwd()
project_path = cwd[:cwd.find('pygents')+7]
if project_path not in sys.path: sys.path.append(project_path)
os.chdir(project_path) 

from importlib import reload  # Python 3.4+

import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

#force reimport
if 'pygents.util' in sys.modules:
    del sys.modules['pygents.util']
if 'pygents.text' in sys.modules:
    del sys.modules['pygents.text']
if 'pygents.plot' in sys.modules:
    del sys.modules['pygents.plot']
if 'pygents.token' in sys.modules:
    del sys.modules['pygents.token']

from pygents.util import * 
from pygents.text import * 
from pygents.plot import * 
from pygents.token import * 

import jieba


In [5]:
# https://www.vengaglobal.com/blog/simplified-traditional-chinese-mandarin-cantonese/

# Target Market  Written      Spoken
# -------------------------------------
# China          Simplified   Mandarin
# Singapore      Simplified   Mandarin
# Taiwan         Traditional  Mandarin
# Hong Kong      Traditional  Cantonese

# Lexicon:
# http://www.chineselexicaldatabase.com/download.php - used below
# Sun, C. C., Hendrix, P., Ma, J.Q. & Baayen, R. H. (2018). Chinese Lexical Database (CLD): A large-scale lexical database for simplified Mandarin Chinese. Behavior Research Methods, https://doi.org/10.3758/s13428-018-1038-3.

# Corpora:
# https://www.openslr.org/38/ - test-audio corpus, not relevant
# https://github.com/CLUEbenchmark/CLUECorpus2020/ - email request sent
# https://github.com/brightmart/nlp_chinese_corpus - nearly same as above downloaded, used further

# TODO:
# https://metatext.io/datasets/nlp-chinese-corpus - paper with word segmentation


In [6]:
path = '../../nlp/corpora/Chinese/'


In [4]:
def zh_clue_json2text(path,filename):
    with open(os.path.join(path,filename+'.json')) as file:
        with open(os.path.join(path,filename+'.txt'), 'w') as fout:
            while True:
                line = file.readline()
                if not line:
                    break
                j = json.loads(line)
                #print('title',j['title'])
                #print('desc',j['desc'])
                #print('content',j['content'])
                fout.write(j['title'])
                fout.write('\n')    
                fout.write(j['desc'])
                fout.write('\n')    
                fout.write(j['content'])
                fout.write('\n')    


In [5]:
#do this once!
#zh_clue_json2text(path,'clue/new2016zh/news2016zh_valid')


In [6]:
#do this once!
#zh_clue_json2text(path,'clue/new2016zh/news2016zh_train')


## Load and explore full models

In [3]:
zh_valid_chars = FreedomTokenizer(max_n=3,mode='chars',debug=False)
#zh_valid_grams = FreedomTokenizer(max_n=3,mode='grams',debug=False)

with open(join(path, 'clue/new2016zh/news2016zh_valid.txt'),errors='ignore') as f:
    cnt = 0
    while True:
        line = f.readline()
        if not line:
            break
        cnt += 1
        if (cnt % 1000) == 0:
            print(cnt)
        zh_valid_chars.train([line])
        #zh_valid_grams.train([line])

zh_valid_chars.store('data/models/zh_valid_chars_3a')
#zh_valid_grams.store('data/models/zh_valid_grams_3a')

print(zh_valid_chars.count_params())
# 143,129,564 (max_n=3)

#print(zh_valid_grams.count_params())


NameError: name 'path' is not defined

In [8]:
del zh_valid_chars

In [7]:
zh_train_chars = FreedomTokenizer(max_n=2,mode='chars',debug=False)

with open(join(path, 'clue/new2016zh/news2016zh_train.txt'),errors='ignore') as f:
    cnt = 0
    while True:
        line = f.readline()
        if not line:
            break
        cnt += 1
        if (cnt % 100000) == 0:
            print(cnt)
        zh_train_chars.train([line])
        #zh_valid_grams.train([line])

zh_train_chars.store('data/models/zh_train_chars_2a')

print(zh_train_chars.count_params())
# 249,859,247 (max_n=2)


100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
249859247


In [25]:
zh_lexicon_tokenizer = LexiconIndexedTokenizer(lexicon = list(pd.read_csv(os.path.join(path,'lexicon/chineselexicaldatabase2.1.txt'))['Word']))


/Users/akolonin/Documents/aigents/pygents/env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3524: DtypeWarning: Columns (3,4,8,9,12,13,17,18,22,23,28,29,111,112,127,128) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [26]:
#'Цвели яблони и груши, над рекой плыл туман.'
text = '苹果树和梨树开花，雾气飘过河面。'
                               
expected = [r[0] for r in jieba.tokenize(text)]
print(expected)

tokens = zh_lexicon_tokenizer.tokenize(text)
print(round(calc_f1(expected,tokens),2),tokens)
    
print()
for nlist in [[1],[1,2],[2]]:
    for threshold in [0.02,0.05,0.1,0.2,0.3,0.4,0.5]: 
        tokens = tokenize_with_opposite_metrics(zh_train_chars.model,text,'ddf-','ddf+',[1],threshold=threshold)
        print(nlist,threshold,round(calc_f1(expected,tokens),2),tokens)

['苹果树', '和', '梨树', '开花', '，', '雾气', '飘过', '河面', '。']
0.84 ['苹果树', '和', '梨树', '开花', '，', '雾气', '飘', '过', '河面', '。']

[1] 0.02 0.5 ['苹果', '树', '和', '梨树', '开', '花', '，', '雾气飘', '过', '河面', '。']
[1] 0.05 0.82 ['苹果树', '和', '梨树', '开花', '，', '雾气飘过', '河面', '。']
[1] 0.1 0.63 ['苹果树', '和', '梨树开花', '，', '雾气飘过', '河面', '。']
[1] 0.2 0.53 ['苹果树', '和', '梨树开花', '，', '雾气飘过河面', '。']
[1] 0.3 0.53 ['苹果树', '和', '梨树开花', '，', '雾气飘过河面', '。']
[1] 0.4 0.31 ['苹果树和梨树开花', '，', '雾气飘过河面', '。']
[1] 0.5 0.31 ['苹果树和梨树开花', '，', '雾气飘过河面', '。']
[1, 2] 0.02 0.5 ['苹果', '树', '和', '梨树', '开', '花', '，', '雾气飘', '过', '河面', '。']
[1, 2] 0.05 0.82 ['苹果树', '和', '梨树', '开花', '，', '雾气飘过', '河面', '。']
[1, 2] 0.1 0.63 ['苹果树', '和', '梨树开花', '，', '雾气飘过', '河面', '。']
[1, 2] 0.2 0.53 ['苹果树', '和', '梨树开花', '，', '雾气飘过河面', '。']
[1, 2] 0.3 0.53 ['苹果树', '和', '梨树开花', '，', '雾气飘过河面', '。']
[1, 2] 0.4 0.31 ['苹果树和梨树开花', '，', '雾气飘过河面', '。']
[1, 2] 0.5 0.31 ['苹果树和梨树开花', '，', '雾气飘过河面', '。']
[2] 0.02 0.5 ['苹果', '树', '和', '梨树', '开', '花', '，', '雾气飘', '过', '河面', '。']

In [34]:
# https://magichub.com/datasets/chinese-english-parallel-corpus-finance/
test_df = pd.read_csv(os.path.join(path,'magicdata/zh_en_ru_100/CORPUS_ZH_EN_RU.txt'),delimiter='\t')


In [50]:
def evaluate_tokenizer(texts,tokenizer):
    avg_f1 = 0
    count = 0
    for text in texts:
        print(text)
        expected = [r[0] for r in jieba.tokenize(text)]
        tokens = tokenizer.tokenize(text)
        f1 = calc_f1(expected,tokens)
        print(round(f1,2),tokens)
        avg_f1 += f1
        count += 1
    print(round(avg_f1/count,2))

evaluate_tokenizer(list(test_df[:5]['zh']),zh_lexicon_tokenizer)


然后医疗保险呢？就是我们家，不论是大人啊还是小孩都会去买一个保险
0.84 ['然后', '医疗', '保险', '呢', '？', '就是', '我们', '家', '，', '不论是', '大人', '啊', '还是', '小孩', '都会', '去', '买', '一个', '保险']
当他们买了保险的，按照保险合同的话，是要赔三十万的
0.8 ['当', '他们', '买', '了', '保险', '的', '，', '按照', '保险', '合同', '的话', '，', '是', '要', '赔', '三', '十', '万', '的']
需要意识到买了一个保险的重要性
1.0 ['需要', '意识', '到', '买', '了', '一个', '保险', '的', '重要性']
其实这种现象是真的很普遍，因为比如说你买一个人身意外险你那个你买的越多你那个保额就越多
0.85 ['其实', '这', '种', '现象', '是', '真的', '很', '普遍', '，', '因为', '比如说', '你', '买', '一个', '人身', '意外', '险', '你', '那个', '你', '买', '的', '越', '多', '你', '那个', '保', '额', '就', '越', '多']
这代父母真的很有必要去买一个保险
0.86 ['这', '代', '父母', '真的', '很', '有', '必要', '去', '买', '一个', '保险']
0.87


In [46]:
for i,row in test_df[:5].iterrows():
    text = row['zh']
    print(text)
    expected = [r[0] for r in jieba.tokenize(text)]
    tokens = zh_lexicon_tokenizer.tokenize(text)
    tokens = tokenize_with_opposite_metrics(zh_train_chars.model,text,'ddf-','ddf+',[1],threshold=0.5)
    print(expected)
    print(tokens)
    print(round(calc_f1(expected,tokens),2))


然后医疗保险呢？就是我们家，不论是大人啊还是小孩都会去买一个保险
['然后', '医疗保险', '呢', '？', '就是', '我们', '家', '，', '不论是', '大人', '啊', '还是', '小孩', '都', '会', '去', '买', '一个', '保险']
['然后医疗保险呢？就是我们家', '，', '不论是大人啊还是小孩都会去买一个保险']
0.09
当他们买了保险的，按照保险合同的话，是要赔三十万的
['当', '他们', '买', '了', '保险', '的', '，', '按照', '保险合同', '的话', '，', '是', '要', '赔', '三十万', '的']
['当他们买了保险', '的', '，', '按照保险合同', '的', '话', '，', '是要赔三十万', '的']
0.32
需要意识到买了一个保险的重要性
['需要', '意识', '到', '买', '了', '一个', '保险', '的', '重要性']
['需要意识到买了', '一个保险', '的', '重要性']
0.31
其实这种现象是真的很普遍，因为比如说你买一个人身意外险你那个你买的越多你那个保额就越多
['其实', '这种', '现象', '是', '真的', '很', '普遍', '，', '因为', '比如说', '你', '买', '一个', '人身', '意外险', '你', '那个', '你', '买', '的', '越', '多', '你', '那个', '保额', '就', '越', '多']
['其实这种现象是真', '的', '很普遍', '，', '因为比如说你买一个人身意外险你那个你买', '的', '越多你那个保额就越多']
0.11
这代父母真的很有必要去买一个保险
['这代', '父母', '真的', '很', '有', '必要', '去', '买', '一个', '保险']
['这代父母真', '的', '很有', '必要去买', '一个保险']
0


In [ ]:
#del zh_train_chars